# IMPORTING LIBRARIES

In [1]:
import pandas as pd
import sqlite3 as sql

# CONNECTION FROM DATABASE TO DATASET

In [2]:
con = sql.connect('factbook.db')
con

# DEFINING FUNCTION TO RUN QUERIES DYNAMICALLY

In [3]:
def run_query(query):
    return pd.read_sql_query(query,con)

# DATASET OF COUNTRIES

In [4]:
query = 'select * from facts'
run_query(query)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,3.256434e+07,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3.029278e+06,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,8.558000e+04,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...
256,257,zh,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,258,xo,Indian Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,259,zn,Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,oo,Southern Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DATASET OF CITIES

In [5]:
query = 'select * from cities'
run_query(query)

,id,name,population,capital,facts_id
0,1,Oranjestad,37000,1,216
1,2,Saint John'S,27000,1,6
2,3,Abu Dhabi,942000,1,184
3,4,Dubai,1978000,0,184
4,5,Sharjah,983000,0,184
...,...,...,...,...,...
392,393,Mbabane,66000,1,167
393,394,Sanaa,2419000,1,193
394,395,Aden,784000,0,193
395,396,Lusaka,1802000,1,194


# ANALYSING DATA (EDA)

In [6]:
query = """ select * from facts 
            where name =='Pakistan' """
run_query(query)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54


In [7]:
query = """ select * from cities 
            where facts_id == 2 """
run_query(query)

,id,name,population,capital,facts_id
0,10,Tirana,419000,1,2


In [8]:
query = """ select * from cities 
            where facts_id == 132 and capital== 1 """
run_query(query)

,id,name,population,capital,facts_id
0,278,Islamabad,919000,1,132


In [9]:
query = """ select facts.name, cities.name from facts 
            left join cities
            on facts.id==cities.facts_id
            where cities.capital== 1
            """
run_query(query)

,name,name
0,Aruba,Oranjestad
1,Antigua and Barbuda,Saint John'S
2,United Arab Emirates,Abu Dhabi
3,Afghanistan,Kabul
4,Algeria,Algiers
...,...,...
203,Samoa,Apia
204,Swaziland,Mbabane
205,Yemen,Sanaa
206,Zambia,Lusaka


In [10]:
query = """ select * from cities
            where population >= 10000000 and
            capital == 1
            """
run_query(query)

,id,name,population,capital,facts_id
0,16,Buenos Aires,13528000,1,7
1,37,Dhaka,15391000,1,14
2,81,Beijing,15594000,1,37
3,111,Cairo,11169000,1,53
4,161,New Delhi,22654000,1,77
5,190,Tokyo,37217000,1,85
6,240,Mexico City,20446000,1,114
7,291,Manila,11862000,1,138
8,296,Moscow,11621000,1,143


# TASK_01

In [11]:
query = """ select facts.name, cities.name, cities.population from facts 
            left join cities
            on facts.id==cities.facts_id
            where cities.population >= 10000000 and
            cities.capital == 1
            """
run_query(query)

,name,name,population
0,Argentina,Buenos Aires,13528000
1,Bangladesh,Dhaka,15391000
2,China,Beijing,15594000
3,Egypt,Cairo,11169000
4,India,New Delhi,22654000
5,Japan,Tokyo,37217000
6,Mexico,Mexico City,20446000
7,Philippines,Manila,11862000
8,Russia,Moscow,11621000


# TASK_02

# DATA FILTERING AND ANALYSING (EDA)

In [12]:
query = """ select sum(population) from cities
            where facts_id == 195
            """
run_query(query)

,sum(population)
0,1542000


In [13]:
query = """
select
(select population from facts)
-

(select sum(cities.population) from cities
            left join facts
            on facts.id==cities.facts_id
            group by facts.name) as urban
        
from facts

"""
run_query(query)

,urban
0,29467342
1,29467342
2,29467342
3,29467342
4,29467342
...,...
256,29467342
257,29467342
258,29467342
259,29467342


In [14]:
query = """select population from facts group by id"""
run_query(query)

,population
0,3.256434e+07
1,3.029278e+06
2,3.954217e+07
3,8.558000e+04
4,1.962535e+07
...,...
256,NaN
257,NaN
258,NaN
259,NaN


# FOR URBAN AREAS

In [16]:
query = """
select facts.name, facts.population, sum(cities.population) as Urban, 
            sum(cities.population)/cast(facts.population as float)*100 as percent
            
            from cities
            right join facts
            on facts.id==cities.facts_id
            group by facts.id"""
run_query(query)

,name,population,Urban,percent
0,Afghanistan,3.256434e+07,3097000.0,9.510403
1,Albania,3.029278e+06,419000.0,13.831679
2,Algeria,3.954217e+07,3699000.0,9.354571
3,Andorra,8.558000e+04,23000.0,26.875438
4,Angola,1.962535e+07,6166000.0,31.418543
...,...,...,...,...
256,Atlantic Ocean,NaN,NaN,NaN
257,Indian Ocean,NaN,NaN,NaN
258,Pacific Ocean,NaN,NaN,NaN
259,Southern Ocean,NaN,NaN,NaN


# FOR RURAL AREAS

In [19]:
query = """
            select facts.name, facts.population, 
            cast(facts.population as float)-sum(cities.population) as Rural, 
            (cast(facts.population as float)-sum(cities.population)) /
            cast(facts.population as float)*100 as percent
            
            from cities
            right join facts
            on facts.id==cities.facts_id
            group by facts.id
        """
run_query(query)

,name,population,Rural,percent
0,Afghanistan,3.256434e+07,29467342.0,90.489597
1,Albania,3.029278e+06,2610278.0,86.168321
2,Algeria,3.954217e+07,35843166.0,90.645429
3,Andorra,8.558000e+04,62580.0,73.124562
4,Angola,1.962535e+07,13459353.0,68.581457
...,...,...,...,...
256,Atlantic Ocean,NaN,NaN,NaN
257,Indian Ocean,NaN,NaN,NaN
258,Pacific Ocean,NaN,NaN,NaN
259,Southern Ocean,NaN,NaN,NaN
